深入理解 Python 中的不同序列类型，不但能让我们避免重新发明轮子，
它们的 API 还能帮助我们把自己定义的 API 设计得跟原生的序列一样，
或者是跟未来可能出现的序列类型保持兼容。

## 2.1 内置序列类型概览

Python 标准库用 C 实现了丰富的序列类型，列举如下：
* 容器序列
list、tuple 和 collections.deque 这些序列能存放不同类型的数据。
* 扁平序列
str、bytes、bytearray、memoryview 和 array.array，这类序列只能容纳一种类型。

容器序列存放的是它们所包含的任意类型的对象的引用，而扁平序列
里存放的是值而不是引用。换句话说，扁平序列其实是一段连续的内存
空间。由此可见扁平序列其实更加紧凑，但是它里面只能存放诸如字
符、字节和数值这种基础类型。


**序列类型还能按照能否被修改来分类；**

* 可变序列：
list、bytearray、array.array、collections.deque 和
memoryview。
* 不可变序列：
tuple、str 和 bytes。

列表推导是一种构建列表的方法，它异常强大，然
而由于相关的句法比较晦涩，人们往往不愿意去用它。掌握列表推导还
可以为我们打开生成器表达式（generator expression）的大门，后者具有
生成各种类型的元素并用它们来填充序列的功能。

## 2.2 列表推导式和生成器表达式

列表推导是构建列表（list）的快捷方式，而生成器表达式则可以用来
创建其他任何类型的序列。

> 很多 Python 程序员都把列表推导（list comprehension）简称为
listcomps，生成式表达器（generator expression）则称为 genexps。

In [9]:
# 普通循环方式
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol))
print(codes)

[36, 162, 163, 165, 8364, 164]


In [10]:
# 列表推导式
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

** Python3中列表推导不会再有变量泄漏的问题**

列表推导、生成器表达式，以及同它们很相似的集合（set）推导
和字典（dict）推导，在 Python 3 中都有了自己的局部作用域，就
像函数似的。表达式内部的变量和赋值只在局部起作用，表达式的
上下文里的同名变量还可以被正常引用，局部变量并不会影响到它
们。

In [14]:
x = 'ABC'
dummy = [ord(x) for x in x]
print(dummy) # 列表推导式也创建了正确的列表
print(x) # x的值被保留了

[65, 66, 67]
ABC


**列表推导同filter和map的比较**

列表推导可以帮助我们把一个序列或是其他可迭代类型中的元素过滤或
是加工，然后再新建一个列表。Python 内置的 filter 和 map 函数组合
起来也能达到这一效果，但是可读性上打了不小的折扣。

filter 和 map 合起来能做的事情，列表推导也可以做，而且还不需要
借助难以理解和阅读的 lambda 表达式。

In [15]:
# 列表推导式
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

In [18]:
# filter, map
beyond_ascii = list(filter(lambda c: c > 127, map(ord, symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

In [20]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors for size in sizes]
tshirts

[('black', 'S'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 'S'),
 ('white', 'M'),
 ('white', 'L')]

In [21]:
for c in colors:
    for s in sizes:
        print((c, s))

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


In [22]:
for s in sizes:
    for c in colors:
        print((s, c))

('S', 'black')
('S', 'white')
('M', 'black')
('M', 'white')
('L', 'black')
('L', 'white')


列表推导的作用只有一个：生成列表。如果想生成其他类型的序列，生成器表达式就派上了用场。

### 2.2.4 生成器表达式

虽然也可以用列表推导来初始化元组、数组或其他序列类型，但是生成
器表达式是更好的选择。这是因为生成器表达式背后遵守了迭代器协
议，可以逐个地产出元素，而不是先建立一个完整的列表，然后再把这
个列表传递到某个构造函数里。前面那种方式显然能够节省内存。

生成器表达式的语法跟列表推导差不多，只不过把方括号换成圆括号而已。

In [23]:
symbols = '$¢£¥€¤'
tuple(ord(symbol) for symbol in symbols)

(36, 162, 163, 165, 8364, 164)

In [27]:
colors = ['black', 'white']
sizes = ['S', 'M', 'L']

for shirt in ((c,s) for c in colors for s in sizes):
    print(shirt)

('black', 'S')
('black', 'M')
('black', 'L')
('white', 'S')
('white', 'M')
('white', 'L')


## 2.3 元组不仅仅是不可变得列表

有些 Python 入门教程把元组称为“不可变列表”，然而这并没有完全概括
元组的特点。除了用作不可变的列表，它还可以用于没有字段名的记
录。鉴于后者常常被忽略，我们先来看看元组作为记录的功用。

### 2.3.1 元组和记录

元组其实是对数据的记录：元组中的每个元素都存放了记录中一个字段
的数据，外加这个字段的位置。正是这个位置信息给数据赋予了意义。

如果只把元组理解为不可变的列表，那其他信息——它所含有的元素的
总数和它们的位置——似乎就变得可有可无。但是如果把元组当作一些
字段的集合，那么数量和位置信息就变得非常重要了。

拆包让元组可以完美地被当作记录来使用。



### 2.3.2 元组拆包

元组拆包可以应用到任何可迭代对象上，唯一的硬性要求
是，被可迭代对象中的元素数量必须要跟接受这些元素的元组的空
档数一致，除非我们用 * 来表示忽略多余的元素。



最好辨认的元组拆包形式就是平行赋值，也就是说把一个可迭代对象里
的元素，一并赋值到由对应的变量组成的元组中。就像下面这段代码：

In [28]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # 元组拆包

In [29]:
latitude

33.9425

In [30]:
longitude

-118.408056

另外一个很优雅的写法当属不使用中间变量交换两个变量的值：

In [34]:
a = 0
b =10
a, b = b, a # 交换a, b的值

print(a)
print(b)

10
0


还可以用 * 运算符把一个可迭代对象拆开作为函数的参数：


In [35]:
divmod(20, 8)

(2, 4)

In [39]:
t = (20, 8)
quotient, remainder = divmod(*t)

print((quotient, remainder))

(2, 4)


下面是另一个例子，这里元组拆包的用法则是让一个函数可以用元组的
形式返回多个值，然后调用函数的代码就能轻松地接受这些返回值。比
如 os.path.split() 函数就会返回以路径和最后一个文件名组成的元
组 (path, last_part):

In [43]:
import os

root, filename = os.path.split("/home/crise/.ssh/idrsa.pub")

In [44]:
(root, filename)

('/home/crise/.ssh', 'idrsa.pub')

除此之外，在元组拆包中使用 * 也可以帮助我们把注意力集中在元组的部分元素上。用*来处理剩下的元素。在 Python 中，函数用 *args 来获取不确定数量的参数算是一种经典写
法了。

In [46]:
a, b, *rest = range(5)
(a, b, rest)

(0, 1, [2, 3, 4])

在平行赋值中，* 前缀只能用在一个变量名前面，但是这个变量可以出现在赋值表达式的任意位置：

In [47]:
a, *body, c, d = range(10)
(a, body, c, d)

(0, [1, 2, 3, 4, 5, 6, 7], 8, 9)

In [48]:
*head, b, c, d = range(10)
(head, b, c, d)

([0, 1, 2, 3, 4, 5, 6], 7, 8, 9)

### 2.3.3 嵌套元组拆包

接受表达式的元组可以是嵌套式的，例如 (a, b, (c, d))。只要这个
接受元组的嵌套结构符合表达式本身的嵌套结构，Python 就可以作出正
确的对应。

In [50]:
metro_areas = [
('Tokyo','JP',36.933,(35.689722,139.691667)), 
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
print('{:15} | {:^9} | {:^9}'.format('', 'lat.', 'long.'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas: 
    if longitude <= 0: 
        print(fmt.format(name, latitude, longitude))

                |   lat.    |   long.  
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


元组已经设计得很好用了，但作为记录来用的话，还是少了一个功能：我们时常会需要给记录中的字段命名。namedtuple 函数的出现帮我们解决了这个问题。

### 2.3.4 具名元组

collections.namedtuple 是一个工厂函数，它可以用来构建一个带字段名的元组和一个有名字的类——这个带名字的类对调试程序有很大帮助。

> 用 namedtuple 构建的类的实例所消耗的内存跟元组是一样的，因为字段名都被存在对应的类里面。这个实例跟普通的对象实例比起来也要小一些，因为 Python 不会用 `__dict__`来存放这些实例的属性。